# DATA MODEL AND OBJECT HIERARCHY DESIGN PRINCIPLES
---
## USERS & PAYMENT METHODS
---

### Basics
1. our application has **users** $\rightarrow$ `User` *abstract class*.
2. our **users** are :
    - **clients**, consuming our services, such as **parents** or **students** $\rightarrow$ `Client` *abstract class*, `Parent` and `Student` *concrete classes*.
    - **providers**, providing material for our services, such as professors $\rightarrow$ `Provider` *abstract class* and `Professor` *concrete class*.
3. to **pay** for the fees for our **services**, one/many **payment methods** can be used $\rightarrow$ `PaymentMethod` *abstract class*
4. a **payment method** can be either a **bank card** or a **bank transfer** $\rightarrow$ `BankCard` and `BankTransfer` *concrete classes*.
5. a **payment method** can be used to make a **payment** $\rightarrow$ `Payment` *concrete class*.

### Details
#### User
1. a `User` has :
    - a `name` ;
    - a `surname` ;
    - an `email` ;
    - a `login` ;
    - a `password` ;
    - a **date of subscription** $\rightarrow$ attribute `dateSubscription`.
2. **table**:

`User` (*abstract class*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | --
**name**             | `String`      |not null              | --
**surname**          | `String`      |not null              | --
**login**            | `String`      |unique and not null   | --
**password**         | `String`      |not null              | --
**email**            | `String`      |unique and not null   | validation côté client en utilisant des regex
**dateSubscription** | `DateTime`    |not null              | datetime courant lors de l'inscription ayant le format DD/MM/YYYY

---

#### Client
1. a `Client` has to **confirm his subscription** by clicking on an email link sent after his **signing up** to the applicaiton $\rightarrow$ `emailVerified` attribute, by default equals **false** and becomes **true** upon clicking on **confirming subscription**.
2. **table**:

`Client` (*abstract class inheriting `User`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | clé étrangère référençant le tuple correspondant dans la superclasse `User`
**emailVerified**    | `boolean`     |not null, by default = `False`              | valeur mise à jour lors de la confirmation par email par le client

---

#### Student
1. a `Student` has a : 
    - **date of birth** $\rightarrow$ attribute `dateBirth`
    - a last session path attribute `lastSessionPath` designating the root folder of the last connection session to the application, containing :
        1. a **global session metadata JSON object** containing access and exit datetimes.
        2. **micro sessions** for each unit interaction, represented in **session metadata JSON objects**.
2. **table**:

`Student` (*concrete class inheriting `Client`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | clé étrangère référençant le tuple correspondant dans la superclasse `Client`
**dateBirth**        | `DateTime`    |not null              | --
**lastSessionPath**  | `String`      |null by defaut              | chemin vers le fichier JSON contenant les métadonnées sur la dernière session d'accès à l'application

---

#### Parent
1. a `Parent` is either a **mother** or a **father** of a **student** $\rightarrow$ attribute `role` equals either **"Mother"** or **"Father"**.
2. a **parent** can have *one/many* **students** as children, while a **student** can only have one **parent** subscribing him to our services $\rightarrow$ `ParentHasChild` aggregation between `Parent` and `Student`.
3. a **parent** can notify *one/many* of its children $\rightarrow$ `ParentNotifiesChild` association having :
    - a datetime of notification attribute `dateNotification`.
    - a message attribute associated to the notification `message`.
3. **tables**:

`Parent` (*concrete class inheriting `Client`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | clé étrangère référençant le tuple correspondant dans la superclasse `Client`
**role**             | `String`      |not null and check value in {`'Mother'`, `'Father'`}| énumération côté client


`ParentHasChild` ($1..N$ binary *association between `Parent` and `Student`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**idPS**             | `int`         |clé primaire          | --
**idP**              | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `Parent`
**idS**              | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `Student`

**Remarques**  
1. (`idP`, `idS`) doit être unique dans la table

---

`ParentNotifiesChild` ($N..N$ binary *association between `Parent` and `Student`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**idPNS**             | `int`         |clé primaire          | --
**idP**               | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `Parent`
**idS**               | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `Student`, qui doit être référencé dans `ParentHasChild`
**dateNotification**      | `DateTime`    |not null              | datetime choisi pour notifier le student, ayant le format DD/MM/YYYY HH:MM
**message**           | `String`          |null par défaut       | message pouvant être associé au notification

**Remarques**  
1. (`idP`, `idS`, `dateNotification`) doit être unique dans la table

---

#### PaymentMethod
1. a `Client` can have *one/many* **payment methods**, while a **payment method** can only be owned by a single **client** $\rightarrow$ `PaymentMethod` **table** references the corresponding **client** (*i.e.* ***parent*** *or* ***student**) by its **id**, `idHolder`.
2. even though a `PaymentMethod` is associated with a **client** in our database, the credentials provided by the **client** for the application (*i.e. `name` and `surname`*) might not be identical to those provided to identify his **payment method** $\rightarrow$ attributes `holderName` and `holderSurname` to designate the credentials of the **client** used to identify his **payment method**.
3. if a **client** is a `Parent`, his **payment methods** will be reverberated to his **children**. However, only the **parent** will have control over these **payments methods** (*read/write access*) whereas the children can only consult them (*read access only*).
4. if a **client** is a `Student`, he will naturally have control over his **payment methods** (*read/write access*).
5. **notice**: there will be no actual payment process in the application, but only a fake process mimicking a real payment.
6. **table**:

`PaymentMethod` (*abstract class*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | --
**holderName**       | `String`      |not null              | --
**holderSurname**    | `String`      |not null              | --
**idHolder**         | `int`         |not null              |clé étrangère référençant un tuple correspondant dans la table `Client`

**Remarques**  
1. la classe correspondante à la table `PaymentMethod` possède une méthode abstraite `pay()` qui sera implémentée par toutes ces sous-classes concrètes

---

#### BankCard
1. a `BankCard` has:
    - a `number` ;
    - an **expiry date** $\rightarrow$ attributes `expiryMonth` and `expiryYear` ;
    - a **security code** $\rightarrow$ attribute `securityCode`.
2. **table**:

`BankCard` (*concrete class inheriting `PaymentMethod`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | clé étrangère référençant le tuple correspondant dans la superclasse `PaymentMethod`
**number**       | `String`      |not null              | validation côté client en utilisant des regex
**expiryMonth**  | `int`         |not null and check value in $[1;12]$              | --
**expiryYear**   | `int`         |not null and check value $\geq$ année courante              | --
**securityCode** | `int`         |not null and check length = 3              | validation côté client

**Remarques**  
1. (`holderName`, `holderSurname`, `number`, `expiryMonth`, `expiryYear`, `securityCode`) doit être unique.

---

#### BankTransfer
1. a `BankTransfer` has:
    - **unique RIB** value associated with the account from which to transfer $\rightarrow$ attribute `rib`.
2. **table**:

`BankTransfer` (*concrete class inheriting `PaymentMethod`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | clé étrangère référençant le tuple correspondant dans la superclasse `PaymentMethod`
**rib**              | `String`      |unique and not null              | validation côté client en utilisant des regex

---
    
#### Payment
1. a **payment** occurs when a **client** uses a **payment method** to pay for the fees of his **courses** for a given school **year** at a given **date**. The payment will be identified by a **reference**, mimicking a **receipt** :
    - a **payment method** can be used to make *one/many* **payments**, whereas a **payment** is only done by a single **payment method**. On ther other hand, `PaymentMethod` references the **owner** of the method (*i.e. the* ***client***), so we only need to reference the **payment method** to get access to the **owner** $\rightarrow$ `Payment` **table** references the corresponding **payment method** by its id `idP` (which transitively references the `Client` owning the **payment method** as well).
    - the **payment** will be identified by a **reference**, mimicking a **receipt** $\rightarrow$ attribute `reference`.
    - a **client** will pay upon subscription for all **courses** chosen for a given school **year** $\rightarrow$ attribute `yearSchool` designating the school **year** for all **courses** to be paid for.
    - a **client** will pay upon subscription for all **courses** chosen for a given school **year** at a given **date** $\rightarrow$ attribute `datePayment`.
    - if the **client** is a `Parent`, we have to see his **children** through `ParentHasChild`. For each **child**, we see the **courses** he subscribed to for a given school **year** through `StudentHasCourse`. For each **course**, we obtain the **price** of the **course** via the `price` attribute of `CourseHasPrice` **table**. `totalFees` will be then the **sum of prices** for each **course** subscribed to by each **child**.
    - if the **client** is a `Student`, we have to see the **courses** he subscribed to for a given **year** through `StudentHasCourse` and repeat the same procedure we repeated for a `Parent` **client** from this step onwards. 
2. **table**:

`Payment` (*concrete class*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | --
**idP**              | `int`         |not null              | clé étrangère référençant le tuple correspondant dans la table `PaymentMethod`
**reference**        | `String`      |not null              | générée lors du paiement par une méthode à créer
**yearSchool**       | `int`         |not null              | désigne l'année des cours à régler (e.g. pour rélger les cours de l'année 2018-2019, `yearSchool = 2019`
**datePayment**      | `DateTime`    |not null              | datetime courant lors du paiement ayant le format DD/MM/YYYY
**totalFees**        | `Real`        |not null              | calculé à partir des prix des cours du client associé à la méthode de paiement

---

## COURSES & UNITS
---
### Basics
1. the application operates by means of **courses** offered by **providers** and consumed by **clients** $\rightarrow$ :
    - `Course` *concrete class*. 
    - **notice**: for an initial version, we'll assume that the creators of the application are the ones who actually offer the courses.
2. every **course** has **metadata** (*i.e.* ***information*** *about the course*) and **content** (*i.e.* ***units***) $\rightarrow$ `Unit` *abstract class*.
3. every `Course` could be either **static** (*i.e. already existent*) or **dynamic** (*i.e. given in real time*) $\rightarrow$ `StaticUnit` and `DynamicUnit` *abstract classes*
4. every `StaticUnit` is : 
    - either provided in its **raw format** (*i.e. without any preprocessing for application adaptation, such as a* ***video***) $\rightarrow$ `RawUnit` *abstract class* and `Video` *concrete class*.
    - or provided in its **mapped format** (*i.e. preprocessed for application adaptation, such as a* ***document****, a* ***presentation****, or an* ***exercise***) $\rightarrow$ `MappedUnit` *abstract class*, `Document`, `Presentation`, `Exercise` *concrete classes*.

### Details
#### Course and Price
1. a `Course` has the following **metadata** :
    - a `title` ;
    - a `class`, designating the **level** of the **course** : {`'6eme'`, `'5eme'`, `'4eme'`, `'3eme'`, `'2nde'`, `'1ere'`, `'0eme'`} (`0eme` $\equiv$ la classe de **Terminale**).
    - a `number`, designating the **number** of the **course** for the given **class** (*starting from 01 and incremented for every course with a different title*)
    - a **unique** `code`, **computed** from its previous **metadata** in the following manner : 
        1. **formula** : first 3 letters of `title` + first letter of `class` + `number`
        1. **example** : a Maths course for $4eme$, having a number of $5$ will have `code = 'Mat405'`.
    - **notice** : **metadata** about a **course** are independent from its **content**: "*A maths course will remain a maths course, regardless of how much its content will change and evolve over the years. It will never become a biology course for instance.*" Hence, a **course**'s **metadata** can be considered as **static**, whereas its **content** and its **dynamic changing** can be stored in **separate tables**.
2. a `Course` has a **content** defined by *one/many* **units** that can change over the **years** (*e.g. addition/modification/removal of units*) $\rightarrow$ `CourseHasUnit` **composition** between `Course` and `Unit` having attribute `yearSchool`, to indicate that a **course** has a **unit** for a specific **year**.
3. a `Course` has a **price** that can change over the **years** $\rightarrow$ :
    - to allow the **price** of the **course** to change over the **years** and preserve the **history of price changes**, we need to create a **table** `Price` having an attribute `value`.
    - next, we need to create an $N..N$ **association** between `Course` and `Price` called `CourseHasPrice` referencing a **course** and a **price**, with attribute `yearSchool`.
4. **notice**: in the initial version :
    - we will only be serving **language courses**, `'French'` and `'English'`.
    - we will consider that the **price** for a **course won't change within the same school year**, and only **between years**. (*simple unique constraint change in the concerned tables to revert it later if we wanted to have change within a single year allowed*)
5. **tables**:

`Course` (*concrete class*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | --
**title**            | `String`      |not null and check value in {`'French'`, `'English'`, $\dots$}              | énumération côté client
**class**            | `String`      |not null and check value in {`'6eme'`, `'5eme'`, `'4eme'`, `'3eme'`, `'2nde'`, `'1ere'`, `'0eme'`}| énumération côté client
**number**           | `int`         |not null              | nombre sur deux chiffres, incrémenté automatiquement à partir de 01 pour chaque cours ayant un intitulé différent
**code**             |`String`       |not null et unique    | `code = title[0:2] + class[0] + number`

---

`Price` (*concrete class*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | --
**value**            | `Real`        |not null and unique   | --

---

`CourseHasPrice` (*N..N association between `Course` and `Price`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**idCP**             | `int`         |clé primaire          | --
**idC**              | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `Course`
**idP**              | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `Price`
**yearSchool**       | `int`         |not null              | désigne l'année scholaire pendant laquelle un cours a le prix (e.g. pour l'année 2018-2019, `yearSchool = 2019`)

**Remarques**  
1. (`idC`, `yearSchool`) doit être unique dans la table $\rightarrow$ change in prices over years only, and not within the same year.

---

`CourseHasUnit` (*N..N association between `Course` and `Unit`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**idCU**             | `int`         |clé primaire          | --
**idC**              | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `Course`
**idU**              | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `Unit`
**yearSchool**       | `int`         |not null              | désigne l'année scholaire pendant laquelle un cours contient l'unité (e.g. pour l'année 2018-2019, `yearSchool = 2019`)

**Remarques**  
1. (`idC`, `idU`, `yearSchool`) doit être unique dans la table.

---

#### Units
1. every `Unit` has a `number`, designating the **number** of the **unit** in the **course** (*e.g. exercise 1, exam 1, document 1, $\dots$*)
2. **table**:

`Unit` (*abstract class*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | --
**number**           | `int`         |not null              | désigne le numéro de l'unité (*e.g. exercise 1, cours 1, vidéo 1,* $\dots$) (*cf. Units File Hierarchy*)

---
#### Static units
1. every `StaticUnit` : 
    - has a `name`, since it already exists, in contrary to a `DynamicUnit` which has a `link` designating its happening (*e.g. a video stream, an application implementing the VoIP protocol such as Skype or Google Hangouts, $\dots$*). The **name** designates that of the **root folder** containing the **unit**'s **resources** (*i.e. files, images, $\dots$*)
    - has a `duration` in **minutes**, which can be **computed** based on its **type** (*e.g. a video unit has a duration in minutes, whereas a document can have an estimated duration based on its number of page for instance, $\dots$*)
    - has a `path` designating its **initial state**. In other words, a **path** towards a **file/link** without any preprocessing to adapt it for the Android application. (*e.g. a* ***document*** *will have initially a* ***pdf*** *representing its initial state,whereas a* ***video****, if online will have a URL, otherwise will have a local filesystem path*)

`StaticUnit` (*abstract class inheriting `Unit`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | clé étrangère référençant le tuple correspondant dans la superclasse `Unit`
**name**             | `String`      |not null              | désigne le nom du dossier correspondant à l'unité, contenant ses ressources associées (fichiers, images, $\dots$)
**duration**         | `int`         |not null              | attributé calculé selon le type de l'unité
**path**         | `String`         |not null              | désigne le chemin vers le fichier représentant l'unité sans aucun traitement (*e.g.* `.pdf` pour un **document/lecture**, lien YouTube d'une **vidéo en ligne** ou lien d'une **vidéo en local**, $\dots$. (*cf. Units File Hierarchy*).

**Remarques**  
1. la classe correspondante à la table `StaticUnit` possède une méthode abstraite `duration()` qui sera implémentée par toutes ces sous-classes concrètes afin d'obtenir la valeur de l'attribut `duration`

---

#### Mapped units
1. every `MappedUnit` has a `mapPath` designating the **path** of the **map** file obtained from **preprocessing the unit** to adapt it for the Android application. (*e.g. documents and presentations will have mapped JSON versions obtained from mining their pdf initial files using an external python script*)

`MappedUnit` (*abstract class inheriting `StaticUnit`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | clé étrangère référençant le tuple correspondant dans la superclasse `StaticUnit`
**mapPath**          | `String`      |not null              | désigne le chemin vers la version transformée du de l'unité après traitement de sa version initiale (*cf. Units File Hierarchy*).

---

#### Raw units
1. **table**:

`RawUnit` (*abstract class inheriting `StaticUnit`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | clé étrangère référençant le tuple correspondant dans la superclasse `StaticUnit`

---

#### Documents
1. a `Document` has:
    - a **pdf document** :
        1. to **download** for local storage (initial version of the document)
        2. to be be mapped to its **dynamic version** as it is being interacted with at **Application Level** (at `mapPath`, `MappedUnit` attribute).
        3. having a **path** (at `path`, `StaticUnit` attribute).
        4. has a specific **number of pages** that defines its **estimated duration** for completion
    - a **duration estimation** :
        1. $1~page = 1~point$
        2. we estimate that a student will take $10$ **minutes** to complete a page.
        3. estimated_completion_duration(Document) in minutes = nbPages(Document) $\times 10$.
    - a **dynamic mapped version** :
        1. **GUI**: (to be added to GUI section later)
            * definition of a standard extensible document layout
            * page flipping using previous/next buttons or left/right swipes.
            * document completion using a completion button
            * document checkpoint : the page number having focus, saved locally (SQLite) and flushed to the server before the client quits the application.
        2. **mapping computation** : before unit becomes available for interaction, we must **only once** :
            * download units as **pdf documents** from specific sources, respecting a specific syntax.
            * **extract resources** from pdf documents (*i.e. textual content, tables, images, $\dots$*)
            * **map the extracted content and resources** into **JSON Map file**.
            * **reference the mapped version of the document** in the **database model**.
        3. **tools** :
            * **pdf content and resources extraction**: 
                1. `PDFMiner` python module: content and resources extraction ;
                2. `pdfimages` Unix system command: completing missing images extraction.
            * **mapping of extracted content and resources to JSON representation**: python functions to map the extracted content into well defined JSON objects.
            * **referencing the JSON representation of a course in the database**: PHP server functions that will check last modified dates of map files to update the database entries accordingly.
2. **table**:

`Document` (*concrete class inheriting `MappedUnit`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | clé étrangère référençant le tuple correspondant dans la superclasse `MappedUnit`
**nbPages**          | `int`         |not null              | désigne le nombre de pages du document en regardant le nombre de pages du fichier pdf initial désignant le document, pointé par `path` hérité de `StaticUnit`

**Remarques**  
1. un `Document` implémente la méthode abstraite `duration()` héritée de `StaticUnit` en suivant l'algorithme suivant : $duration = nbPages \times 10$

---

#### Presentations
1. a `Presentation` :
    - has the **same behavior** as a `Document`.
    - can have a **Powerpoint/Impress/KFormat presentation file** associated with it for **local download** through a **path attribute**.
2. **table**:

`Presentation` (*concrete class inheriting `Document`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | clé étrangère référençant le tuple correspondant dans la superclasse `Document`
**presPath**     | `String`      |not null              | désigne le chemin vers le fichier de la présentation (*cf. Units File Hierarchy*)

---

#### Exercises
1. an `Exercise`:
    - has a **category** that designates whether it's a **training exercise** or an **exam**.
    - has a **type** that designates whether it's a **MCQ**, a **Q&A**, $\dots$.
    - has a **duration estimation** :
        1. $1~question = 1~point$
        2. we estimate that a student will take : 
            - $5$ **minutes** to complete a question if the type of the exercise was **MCQ**.
            - $10$ **minutes** to complete a question if the type of the exercise was **QandA**.
        3. estimated_completion_duration(Exercise) in minutes = nbQuestions(Exercise) $\times$ estimated_duration_according_type(Question).

2. **table**:

`Exercise` (*concrete class inheriting `MappedUnit`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | clé étrangère référençant le tuple correspondant dans la superclasse `MappedUnit`
**category**         | `String`      |not null and check value in {`'Training'`, `'Exam'`}              | désigne la catégorie d'un exercise (*cf. Units Design Principles*)
**type**             | `String`      |not null and check value in {`'MCQ'`, `'QandA'`}              | énumération côté client désignant le type d'un exercise
**nbQuestions**          | `int`         |not null              | désigne le nombre de questions de l'exercise en regardant le nombre d'entrées questions du fichier map, pointé par `mapPath` hérité de `MappedUnit`.

**Remarques**  
1. un `Exercise` implémente la méthode abstraite `duration()` héritée de `StaticUnit` selon son type :
    - if type = `MCQ`: $duration = nbQuestions * 5$.
    - if type = `QandA` $duration = nbQuestions * 10$.

---

#### Videos
1. a `Video` :
    - has his **path** attribute (*inherited from `StaticUnit`*) containing the **link** to the video (**local video** or on a video hosting platform such as **YouTube**)
    - implements **duration** as equivalent to the actual duration of the video in number of minutes

---

`Video` (*concrete class inheriting `Lecture`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | clé étrangère référençant le tuple correspondant dans la superclasse `RawUnit`

**Remarques**  
1. un `Video` implémente la méthode abstraite `duration()` héritée de `StaticUnit` en suivant l'algorithme suivant : $duration = nbMins(video)$

---

### Students, courses and units associations
1. a `Student` can have *one/many* **courses** for a given school **year**, and a **course** can be taken by *zero/many* **students** for a given school **year** $\rightarrow$ `StudentHasCourse` :
    - to allow a student to take the **same course more than once over the years**, *if required to*, the `StudentHasCourse` **association** will reference :
        1. the **student** `id`
        1. the **course's metadata** `id`
    - a `yearSchool` attribute will be added to indicate which **year** the **student** subscribed to the **course**. This information will be used to get the **course's content** and **price** by checking their `yearSchool` in `CourseHasPrice` and `CourseHasUnit` **tables**.
    - a `score` attribute will be **computed** as the **means over 20 of all exam and training units' scores** to obtain the **overall final grade** a **student** gets for a **course** at the end of a school **year**.
    - a `isValidated` attribute which will be **computed** based on the **student's score** once the school **year** has ended.
2. a `Student` can consult *one/many* **static units** of a **course** for a given school **year**, similarly $\rightarrow$ `StudentConsultsStaticUnit` $N..N$ **association** between `StudentHasCourse` and `StaticUnit` having:
    - an **id** referencing a `StudentHasCourse` id, and by extension both the **student** and the **course** he's subscribed to for a given school **year**.
    - an **id** referencing a `CourseHasUnit` id, which will be **specialized** into the **id** of one of the **concrete tables** inheriting from `StaticUnit` when the type of the **static unit** is know at runtime.
    - a **completion status** attribute `isCompleted` indicating whether the **unit** is **completed or not** (*by default false*), obtained from the **session metadata JSON object**, after the client confirms unit completion or abandons it without completion and quits the session.
    - an *abstract* **checkpoint attribute** `checkpoint`, which will be **computed** by one of the **specialized static units**, based on their implementation of the `checkpoint()` *abstract method* of the `Checkpointable` **interface** used in the **object class model** on the client, obtained from the **session metadata JSON object**. (won't be implemented explicitly in the table, but used as a placeholder to demonstrate relationship with inheriting tables).
    - a **last access date** attribute `dateLastAccess`, designating the **last datetime** at which the **static unit** gained **student focus** on the app, obtained from the **session metadata JSON object**.
    - a **last session path** attribute `lastSessionPath`, designating the **path** to the **latest session metadata JSON object** stored for the **static unit**.
3. a `Student` can consult *one/many* documents of a course for a given school **year** $\rightarrow$ `StudentConsultsDocument` $N..N$ **association** between `Student` and `Document`, which inherits the attributes of the `StudentConsultsStaticUnit` **association**, and specializes them as follows:
    - an **id** referencing the `Document`, which will **specialize** the **static unit** id referenced in the `CourseHasUnit` table, referenced by an id in `StudentConsultsStaticUnit`.
    - a **checkpoint** designating the number of the last page of the document having user focus before exiting session.
    - **notice**: a presentation is consulted in the same manner as a document, and is referenced therefore in the same consultation table `StudentConsultsDocument`.
4. a `Student` can consult *one/many* exercises of a course for a given school **year** $\rightarrow$ `StudentConsultsExercise` $N..N$ **association** between `Student` and `Exercise`, which inherits the attributes of the `StudentConsultsStaticUnit` **association**, specializes them and adds extra attribute as follows:
    - implemented attributes:
        1. an **id** referencing the `Exercise`, which will **specialize** the **static unit** id referenced in the `CourseHasUnit` table, referenced by an id in `StudentConsultsStaticUnit`.
        2. a **null checkpoint**: exercises are not checkpointable.
    - added attributes:
        1. a **score** attribute, which will be implemented as the **sum of the correct answers** provided for the questions of the exercise, *by default = 0* prior to any client access.
5. a `Student` can consult *one/many* videos of a course for a given school **year** $\rightarrow$ `StudentConsultsVideo` $N..N$ **association** between `Student` and `Video`, which inherits the attributes of the `StudentConsultsStaticUnit` **association**, and specializes them as follows:
    - an **id** referencing the `Video`, which will **specialize** the **static unit** id referenced in the `CourseHasUnit` table, referenced by an id in `StudentConsultsStaticUnit`.
    - a **checkpoint** designating the minutes:second moment at which the video was paused or completed, obtained from the session metadata JSON object, after the client confirms video completion or abandons it without completion and quits the session.
    
6. **tables**:

`StudentHasCourse` ($N..N$ *binary association between `Student` and `Course`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**idSC**             | `int`         |clé primaire          | --
**idS**              | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `Student`
**idCo**             | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `Course`
**yearSchool**       | `int`         |not null              | désigne l'année dans laquelle les cours seront terminés (e.g. pour l'année 2018-2019, `yearSchool = 2019`)
**score**            | `Real`       |--                    | la moyenne des scores de tous les examens du cours sur $20$
**isValidated**      | `boolean`      | by default null, `'Yes'` si `score >= 10`, `'No'` sinon | -- 

**Remarques**  
1. (`idS`, `idCo`, `yearSchool`) doit être unique dans la table

---

`StudentConsultsStaticUnit` ($N..N$ *binary association between `StudentHasCourse` and `CourseHasUnit`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**idSCSU**            | `int`         |clé primaire          | --
**idSC**              | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `StudentHasCourse`
**idCU**             | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `CourseHasUnit`
**isCompleted**      | `boolean`      | not null, by default `'No'`, `'Yes'` si completion confirmée par le bouton | --
**dateLastAccess**      | `DateTime`    |not null              | datetime du dernier accès à l'unité du cours, ayant le format DD/MM/YYYY
**lastSessionPath**          | `String`      |null par défaut              | désigne le chemin vers le fichier contenant les métadonnées de la dernière session d'accès à l'unité sous forme d'un fichier JSON (*cf. Units File Hierarchy*).

**Remarques**  
1. (`idSC`, `idCU`,) doit être unique dans la table
2. les entrées de cette tables seront mises à jour au fur et à mesure des interactions de l'utilisateur avec des static units.

---

`StudentConsultsDocument` ($N..N$ *binary association between `StudentHasCourse` and `CourseHasUnit`, inheriting from `StudentConsultsStaticUnit` and specializing `StaticUnit` into a `Document`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**idSCD**            | `int`         |clé primaire          | clé étrangère référençant un tuple correspondant dans la table `StudentConsultsStaticUnit`
**checkpoint**              | `int`         |null par défaut              | numéro de la dernière page du document ayant le focus de l'utilisateur avant la fin de la session d'interaction

**Remarques**  
1. les entrées de cette tables seront mises à jour au fur et à mesure des interactions de l'utilisateur avec des documents.

---

`StudentConsultsVideo` ($N..N$ *binary association between `StudentHasCourse` and `CourseHasUnit`, inheriting from `StudentConsultsStaticUnit` and specializing `StaticUnit` into a `Video`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**idSCV**            | `int`         |clé primaire          | clé étrangère référençant un tuple correspondant dans la table `StudentConsultsStaticUnit`
**checkpoint**              | `Time`         |null par défaut              | moment minutes:secondes de la vidéo  avant la fin de la session d'interaction

**Remarques**  
1. les entrées de cette tables seront mises à jour au fur et à mesure des interactions de l'utilisateur avec des vidéos.

---

`StudentConsultsExercise` ($N..N$ *binary association between `StudentHasCourse` and `CourseHasUnit`, inheriting from `StudentConsultsStaticUnit` and specializing `StaticUnit` into a `Exercise`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**idSCE**            | `int`         |clé primaire          | clé étrangère référençant un tuple correspondant dans la table `StudentConsultsStaticUnit`
**score**            | `Real`       |--                    | score sur $20$, calculé en comptant le nombre de questions avec des réponses correctes et normalisant sur $20$.
**isValidated**      | `boolean`      | by default null, `'Yes'` si `score >= 10`, `'No'` sinon | -- 

**Remarques**  
1. les entrées de cette tables seront mises à jour au fur et à mesure des interactions de l'utilisateur avec des vidéos.
2. les scores des exercises de catégorie `Exam` sont utilisés pour calculer le score du cours

---
### Recommendations
1. every **student** gets *zero/many* **recommendation** for a specific school **year** at a given **date** :
    - `StudentGetsRecommendation` $0..N$ **association** between `Student` and `Course`, having attributes :
        1. `idS`, referencing the `Student` getting the **recommendation**.
        2. `idC`, referencing the `Course` the `student` is getting the **recommendation** for.
        3. `dateRecomm`, a **datetime** designating the moment the **recommendation** was issued to the **student**.
        4. `yearSchool`, designating the school **year** in which the **recommendation** was issued to the **student**.
        5. `type`, an **enumeration** designating the **type** of the issued **recommendation** ($\in$ {`'General'`, `'Specific'`}).
        6. `recommendation`, a **string** containing the **recommendation comments**, **computed** based on the **type** of the **recommendation**.
    - a **recommendation** could be **general**, computed from the **course's completion percentage**: 
        1. **completion percentage** `P = (number of course's completed units / total number of course's units) * 100`
        2. `if P < 25%`, `comment = "(P% completed) lagging behind and needs to dedicate more time for the course"` ;
        3. `if 25%` $\leq$ `P < 50%, comment = "(_P_% completed) pushing forward, but still not half way there"` ;
        4. `if 50%` $\leq$ `P < 75%, comment = "(_P_% completed) steady steps, keep the good job going!"` ;
        5. `if 75%` $\leq$ `P < 90%, comment = "(_P_% completed) almost there, not very far from achieving the goal, bravo!"` ;
        6. `if P` $\geq$ `90%, comment = "(_P_% completed) Excellent job! Should find no trouble getting great scores"`.
    - a **recommendation** could be **specific**, computed from :
        1. the **completion percentage** `P` (*i.e. the same one used to compute* ***general recommendations***): `comment = "(_P_% completed)"`.
        2. for each **completed static unit** (*after last recommendation date in the recommendations table, if it exists*), a comparison between the **estimated duration** of the **static unit** (`ETC`) (*i.e. the `duration` attribute of `StaticUnit`*) and the **actual duration** of completion (`ATC`) (*obtained by getting the mean value of duration from all sessions associated with the unit, using unit* ***session metadata JSON objects***):
            * if `ATC` $\leq$ `ETC`, `comment += "Unit: excellent timing, completed in _duration_ seconds (_estimated_duration_ seconds required)\n"`.
            * else `comment += "Unit: taking a little too much time, completed in _duration_ seconds (_estimated_duration_ seconds required)\n"`.
        3. for each **completed exercise** in particular (*after last recommendation date in the recommendations table, if it exists*), a comparison between the **full score** of the exercise and the **actual score** on the exercise (*obtained by getting the mean value of score from all sessions associated with the exercise, using* ***exercise session metadata JSON objects***):
            * if `score` $\leq$ `median(totalScore)`, `comment += "Unit: not good enough, needs more practice (_score_/_totalScore_)\n"`.
            * else `score > median(totalScore)`, `comment += "Unit: good job! (_score_/_total_score_)\n"`.
2. **recommendations**:
  - are **computed dynamically** from **client session data**/or **server session data**, upon **request** from a **client** (*parent or student*).
  - are displayed in **reverse chronological order** (*i.e. from the most recent recommendation to the most distant recommendation*).
  - once **computed**, are **stored in the database according to the data model** to preserve the **history of student recommendations** and can be used in later versions for **progress analysis** and **student responsivity to recommendations**.
3. **table**:

`StudentGetsRecommendation` ($0..N$ *binary association between `Student` and `Course`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**idSRC**            | `int`         |clé primaire          | --
**idS**              | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `Student`
**idC**             | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `Course`
**dateRecomm**      | `DateTime`    |not null              | datetime de la recommendation ayant le format DD/MM/YYYY HH:MM
**yearSchool**       | `int`         |not null              | désigne l'année dans laquelle la recommendation a été issue (e.g. pour l'année 2018-2019, `yearSchool = 2019`)
**type**             | `String`      |not null and check value in {`'General'`, `'Specific'`} | énumération côté client, utilisé pour déterminer la manière d'obtenir la valeur de la recommencation.
**recommendation**   | `String`      |not null              | désigne les commentaires de la recommendation (générale ou spécifique), obtenus à partir du type de la recommendation

# CLASS DIAGRAM
---
!["Class diagram of project"](UML/class_diagram.png "Class diagram of project")

# JSON
---
## Server JSON Map Files
---
1. all mapped units will have equivalent JSON map files on the server, that will get fetched by the server from the file system and sent to the client upon request, along with their corresponding metadata from the database and session metadata JSON files.
2. every mapped data will have an JSON map file according to its type.

### Documents

```json
[
    {
        'page': /*int*/,
        'content': /*string, containing references to the position of images*/,
        'images': ['imagePath1', 'imagePath2', ...] /*array of strings, designating each the path of an image on the filesystem*/
    },
    ...
]
```

### Presentations
*same as documents*

### Exercises
#### MCQ

```json
[
    {
        'id': /*int*/,
        'question': /*string*/,
        'answer': /*string*/,
        'choices': ['choice1', ...] /*array of strings*/
    },
    ...
]
```

#### Q & A

```json
[
    {
        'id': /*int*/,
        'question': /*string*/,
        'answer': /*string*/
    },
    ...
]
```

## Client JSON Session Files
1. a **session** will get created **locally** every time a student connects to the application. The session will be represented by a **root directory** having the following **name format** `"session_number_access_date"`.
2. the **session folder** will contain :
    - a **global session metadata JSON object**, containing : 
        1. the **access datetime** and 
        2. the **exit datetime**, initially empty and gets updated when the student quits the application, marking the end of the global session.
    - a **unit session metadata JSON object** for every **unit interaction session**, having the following **name format** `"session_number_unit_title_access_date"`.
3. the **unit session metadata JSON object** will have **different formats** according to **unit type**.
4. if a student interacts with the same unit more than once in the same global application session, different sesion metadata JSON objects will be created for the different unit session interactions.

### Global session metadata JSON object
```json
[
    {
        'access': /*datetime having format DD/MM/YYYY*/,
        'exit': /*datetime having format DD/MM/YYYY, initially empty and updated at the end of the session*/
    }
]
```

### Documents session metadata JSON object
```json
[
    {
        'access': /*datetime having format DD/MM/YYYY*/,
        'exit': /*datetime having format DD/MM/YYYY, initially empty and updated at the end of the session*/,
        'isCompleted': /*boolean, false by default and possibly updated at the end of the session, if document is completed*/,
        'checkpoint': /*int designating page number, initially empty and updated at the end of the session*/
    }
]
```

### Presentations session metadata JSON object
*same as documents*

### Videos session metadata JSON object
```json
[
    {
        'access': /*datetime having format DD/MM/YYYY*/,
        'exit': /*datetime having format DD/MM/YYYY, initially empty and updated at the end of the session*/,
        'isCompleted': /*boolean, false by default and possibly updated at the end of the session, if document is completed*/,
        'checkpoint': /*datetime with format HH:MM:SS, initially empty and updated at the end of the session*/
    }
]
```

### Exercises session metadata JSON objects
#### MCQ
```json
[
    {
        'access': /*datetime having format DD/MM/YYYY*/,
        'exit': /*datetime having format DD/MM/YYYY, initially empty and updated at the end of the session*/,
        'isCompleted': /*boolean, false by default and possibly updated at the end of the session, if document is completed*/,
        'score': /*int, initially empty and updated at the end of the session*/,
        'isValidated': /*boolean, initially empty and updated at the end of the session based on total score*/
    }, /* HEADER OBJECT*/
    { /*CONTENT OBJECTS*/
        'id': /*int*/,
        'question': /*string*/,
        'answer': /*string*/,
        'choices': ['choice1', ...] /*array of strings*/
        'response': /*string*/,
        'score': /*int*/
    },
    ...
]
```

#### Q & A
```json
[
    {
        'access': /*datetime having format DD/MM/YYYY*/,
        'exit': /*datetime having format DD/MM/YYYY, initially empty and updated at the end of the session*/,
        'isCompleted': /*boolean, false by default and possibly updated at the end of the session, if document is completed*/,
        'score': /*int, initially empty and updated at the end of the session*/,
        'isValidated': /*boolean, initially empty and updated at the end of the session based on total score*/
    }, /* HEADER OBJECT*/
    { /*CONTENT OBJECTS*/
        'id': /*int*/,
        'question': /*string*/,
        'answer': /*string*/,
        'response': /*string*/,
        'score': /*int*/
    },
    ...
]
```

# FILE HIERARCHY
---
## Server
---
### Users

### Classes & Courses
**"classes"** (*root folder of all classes*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;class&gt;"** (*a folder for every class in classes*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;course_code&gt;"** (*folder of a class course*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**&lt;year&gt;** (*course version for a given school year*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"documents"** (*folder for all course documents*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_doc&gt;&lt;title_doc&gt;"** (*folder for every document in documents*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_doc&gt;- &lt;title_doc&gt;.pdf"** (*initial version of document file*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_doc&gt;- &lt;title_doc&gt;.json"** (*mapped version of document file*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"images"** (*associated images folder*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_page&gt;- img_&lt;num_img&gt;".k** (*image file, .png, .jpeg, .jpg, .gif*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\dots$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"presentations"** (*folder for all course presentations*): same as **"documents"** + **"&lt;num_doc&gt;- &lt;title_doc&gt;.k"** (*presentation file for every presentation in presentations, .ppt, .odp, $\dots$*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"videos"** (*folder for all course local videos*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_vid&gt;&lt;title_vid&gt;.k"** (*video file, .mp4, .avi, .mkv; $\dots$*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\dots$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"exercises"** (*folder for all course exercises*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"training"** (*folder for all training exercises*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"QandA"** (*folder for all Q&A training exercises*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_qa&gt;&lt;title_qa&gt;"** (*folder for every Q&A training exercise*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_ex&gt;- &lt;title_ex&gt;.pdf"** (*initial version of exercise file*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_ex&gt;- &lt;title_ex&gt;.json"** (*mapped version of exercise file*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"images"** (*associated images folder*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_page&gt;- img_&lt;num_img&gt;".k** (*image file, .png, .jpeg, .jpg, .gif*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\dots$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"MCQ"** (*folder for all MCQ training exerises*): same as **"QandA"**.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"exams"** (*folder for all training exercises*): same as **"training"**

## Client
---
### Users

### Classes & Courses
**"classes"** (*root folder of all classes taken by student*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;class&gt;"** (*a folder for every class in classes*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;course_code&gt;"** (*folder of a class course*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**&lt;year&gt;** (*course version for a given school year, if the student took the course more than once, having failed it the first time*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"documents"** (*folder for all course documents*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_doc&gt;&lt;title_doc&gt;"** (*folder for every document in documents*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:blue">**"&lt;num_doc&gt;- &lt;title_doc&gt;.pdf"**</span> (*initial version of document file, only available if user downloaded the pdf version from the server*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_doc&gt;- &lt;title_doc&gt;.json"** (*mapped version of document file, necessarily present for application offline mode*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"images"** (*associated images folder*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_page&gt;- img_&lt;num_img&gt;".k** (*image file, .png, .jpeg, .jpg, .gif*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\dots$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"sessions"** (*associated sessions folder*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_session&gt;- session_&lt;num_document&gt;_&lt;session$\_$start$\_$datetime&gt;".json** (*session json map, created for every session of interaction with the document*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\dots$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"presentations"** (*folder for all course presentations*): same as **"documents"** + <span style="color:blue">**"&lt;num_doc&gt;- &lt;title_doc&gt;.k"**</span> (*presentation file for every presentation in presentations, .ppt, .odp, $\dots$*, only available if user downloaded the presentation file from the server)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"videos"** (*folder for all course local videos, only contains local videos downloaded from the server*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:blue">**"&lt;num_vid&gt;&lt;title_vid&gt;"**</span> (*folder for every video in video, only available if corresponding local video was downloaded from the server*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:blue">**"&lt;num_vid&gt;&lt;title_vid&gt;.k"**</span> (*video file, .mp4, .avi, .mkv; $\dots$*, only available if user downloaded the video from the server)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\dots$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"sessions"** (*associated sessions folder*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_session&gt;- session_&lt;num_vid&gt;_&lt;session$\_$start$\_$datetime&gt;".json** (*session json map, created for every session of interaction with the video*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\dots$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"exercises"** (*folder for all course exercises*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"training"** (*folder for all training exercises*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"QandA"** (*folder for all Q&A training exercises*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_qa&gt;&lt;title_qa&gt;"** (*folder for every Q&A training exercise*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:blue">**"&lt;num_ex&gt;- &lt;title_ex&gt;.pdf"**</span> (*initial version of exercise file, only available if user downloaded the pdf version from the server*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_ex&gt;- &lt;title_ex&gt;.json"** (*mapped version of exercise file, necessarily present for application offline mode*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"images"** (*associated images folder*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_page&gt;- img_&lt;num_img&gt;".k** (*image file, .png, .jpeg, .jpg, .gif*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\dots$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"sessions"** (*associated sessions folder*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"&lt;num_session&gt;- session_&lt;num_ex&gt;_&lt;session$\_$start$\_$datetime&gt;".json** (*session json map, created for every session of interaction with the exercise*)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\dots$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"MCQ"** (*folder for all MCQ training exerises*): same as **"QandA"**.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**"exams"** (*folder for all training exercises*): same as **"training"**

# USE CASES
---
## Student profile
---
### Sign-up

### Sign-in

### Homepage navigation

### Course navigation

### Unit interaction
#### Document
if the student chooses to interact with a document:
1. the app consults the local storage to see whether or not there are local session information related to the document
2. if this is the first access to the document :
    - the application will create a session JSON file on the local storage, where it will initialize the access date to now, the exit date to nothing, the checkpoint to zero, the completion information to zero
    - it will then fetch from the server the document representation, with which the user will interact.
    - as long as the document has client focus, the session will run. Once the client decides to quit, he will be prompted for confirmation.
    - upon confirmation, a progress bar view will be loaded, while :
        1. his local session JSON entries of exit date, checkpoint and completion will be updated
        2. the updated session JSON file will be flushed to the server, where it will be used later for progress diagrams and recommendations
3. if this is not the first access to the document :
    - the application will fetch from the last session, the checkpoint and completion information
    - it will then fetch from the server the document representation, with which the user will interact.
    - everything else is identical to use case at point "b"

#### Presentation
*same as documents*

#### Video

#### Training Exercises
##### Multiple Choice Questions (MCQ)

##### Questions & Answers (QandA)

#### Exam Exercises
##### Multiple Choice Questions (MCQ)

##### Questions & Answers (QandA)

### Progress diagrams consultation
#### Plots

#### Openclassrooms completion style diagrams

### Recommendation conslutation

### Activity history consultation

### Activity duration plots consultation

### Reminders consultation

### Payment history consultation

### Payment methods consultation

### Offline available functionalities

---

## Parent profile
### Sign-up

### Sign-in

### Homepage navigation

### Progress diagrams consultation
#### Plots

#### Openclassrooms completion style diagrams

### Recommendation conslutation

### Activity history consultation

### Activity duration plots consultation

### Reminders setting

### Payment history consultation

### Payment methods consultation

### Offline available functionalities

# EXTRAS (later)
---
1. `Professor`: concrete class, inheriting `User`, designating a professor user.
2. `ProfessorTeachersCourse`: a $N..N$ association between a `Professor` and a `Course`.
3. `ProfessorConsultsUnit`: a $1..N$ association between a `Professor` and a `Unit`.
4. `DynamicUnit` class hierarchy: for real time course units
5. `RealTimeVideo`: concrete class, inheriting `DynamicUnit`, designating real time video units
6. `Chat` class hierarchy: for premium formula

`Professor` (*concrete class inheriting `User`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**id**               | `int`         |clé primaire          | clé étrangère référençant le tuple correspondant dans la superclasse `User`
**domain**           | `String`      |not null and check value in {`'French'`, `'English'`, `'Spanish'`, $\dots$}| énumération côté client
**degree**           | `String`      |not null and check value in {`'Licence'`, `'Masters'`, `'Phd'`}| énumération côté client

---

`ProfessorTeachesCourse` ($N..N$ *binary association between `Professor`, and `Course`*)

Attribut             | Type          | Contraintes          | Remarques
---------------------|---------------|----------------------|-----------
**idPC**             | `int`         |clé primaire          | --
**idP**              | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `Professor`
**idCo**             | `int`         |not null              | clé étrangère référençant un tuple correspondant dans la table `Course`
**yearSchool**       | `int`         |not null              | --

**Remarques**  
1. (`idP`, `idCo`, `yearSchool`) doit être unique dans la table
2. on assume qu'un professeur apprend toutes les unités d'une matière, et que ses unités ne peuvent pas être partagées par plusieurs professeurs (i.e. un seul professeur pour les CM, les TD, les vidéos, etc.)
